In [1]:
from mlscorecheck.core import load_solutions

In [2]:
solutions = load_solutions()

In [3]:
solutions.keys()

dict_keys([('acc', 'sens'), ('acc', 'spec'), ('acc', 'ppv'), ('acc', 'npv'), ('acc', 'ba'), ('acc', 'f1p'), ('sens', 'spec'), ('ppv', 'sens'), ('npv', 'sens'), ('ba', 'sens'), ('f1p', 'sens'), ('ppv', 'spec'), ('npv', 'spec'), ('ba', 'spec'), ('f1p', 'spec'), ('npv', 'ppv'), ('ba', 'ppv'), ('f1p', 'ppv'), ('ba', 'npv'), ('f1p', 'npv'), ('ba', 'f1p')])

In [4]:
solutions['solutions'][0]


KeyboardInterrupt



In [ ]:
from mlscorecheck.core import base_score_objects, sage_symbols


In [ ]:
def _collect_denominators_and_bases(expression, denoms, bases):
    num = expression.numerator()
    denom = expression.denominator()

    if denom is not None and not denom.is_trivially_equal(1):
        denoms.append(denom)
        _collect_denominators_and_bases(denom, denoms, bases)
    if num.operator() is not None:
        if hasattr(num.operator(), '__qualname__') and num.operator().__qualname__ == 'pow':
            base, exponent = num.operands()
            if exponent < 1:
                bases.append(base)
            _collect_denominators_and_bases(base, denoms, bases)
        else:
            for operand in num.operands():
                _collect_denominators_and_bases(operand, denoms, bases)
    return

def collect_denominators_and_bases(expression):
    denoms = []
    bases = []
    _collect_denominators_and_bases(expression, denoms, bases)
    return denoms, bases

In [ ]:
class ProblemSolver2:
    def __init__(self, score0, score1):
        self.score0 = score0
        self.score1 = score1

    def solve_(self):
        self.solutions = []

        first_equation = self.score0.equation_polynomial
        second_equation = self.score1.equation_polynomial
        first_var = 'tp'
        second_var = 'tn'

        args0 = self.score0.args
        args1 = self.score1.args

        vars0 = int('tp' in args0) + int('tn' in args0)
        vars1 = int('tp' in args1) + int('tn' in args1)

        #print(vars0, vars1)

        if vars0 == 1:
            first_equation = self.score0.equation_polynomial
            second_equation = self.score1.equation_polynomial
            if 'tp' in args0:
                first_var = 'tp'
                second_var = 'tn'
            else:
                first_var = 'tn'
                second_var = 'tp'
        elif vars1 == 1:
            first_equation = self.score1.equation_polynomial
            second_equation = self.score0.equation_polynomial
            if 'tp' in args1:
                first_var = 'tp'
                second_var = 'tn'
            else:
                first_var = 'tn'
                second_var = 'tp'

        #print(first_var, first_equation)
        #print(second_var, second_equation)

        v0s = solve(first_equation, sage_symbols[first_var], solution_dict=True)
        #v0s = sp.solve(first_equation, symbols[first_var], dict=True)
        #print('v0s', v0s)
        for v0 in v0s:

            second_equation_tmp = second_equation
            #if symbols[first_var] in second_equation.free_symbols:
            #print(first_var, second_equation_tmp.args())
            if sage_symbols[first_var] in second_equation.args():
                second_equation_tmp = second_equation.subs(v0)
                #print('subs1')
                #print('se', second_equation_tmp.args())

            v1s = solve(second_equation_tmp, sage_symbols[second_var], solution_dict=True)
            #print('v1sol', second_var, v1s[0][sage_symbols[second_var]].args())
            #v1s = sp.solve(second_equation_tmp, symbols[second_var], dict=True)
            #print('v1s', v1s)
            for v1 in v1s:
                sol = {}
                v0_sol = v0[sage_symbols[first_var]]
                #if sage_symbols[second_var] in v0_sol.free_symbols:
                if sage_symbols[second_var] in v0_sol.args():
                    #print('subs2')
                    v0_sol = v0_sol.subs(v1)

                #print('v0', v0_sol.args())
                #print('v1', v1[sage_symbols[second_var]].args())

                sol[first_var] = v0_sol
                sol[second_var] = v1[sage_symbols[second_var]]
                self.solutions.append(sol)

        return self

    def edge_cases(self):
        self.denoms = []
        self.bases = []
        for solution in self.solutions:
            denoms_sol = set()
            bases_sol = set()
            for _, item in solution.items():
                denoms, bases = collect_denominators_and_bases(item)
                denoms_sol = denoms_sol.union(set(denoms))
                bases_sol = bases_sol.union(set(bases))
            self.denoms.append(denoms_sol)
            self.bases.append(bases_sol)

        return self

    def groebner_basis(self):
        A = PolynomialRing(QQ, names=(sage_symbols['p'], sage_symbols['n'], self.score0.symbol, self.score1.symbol))
        F = A.fraction_field()
        F.inject_variables()
        R = PolynomialRing(F, order='lex', names=(sage_symbols['tp'], sage_symbols['tn'], sage_symbols['fp'], sage_symbols['fn']))
        I = R.ideal(self.score0.equation_polynomial,
                    self.score1.equation_polynomial,
                    sage_symbols['p'] - sage_symbols['tp'] - sage_symbols['fn'],
                    sage_symbols['n'] - sage_symbols['tn'] - sage_symbols['fp'])
        self.gb = I.groebner_basis()
        return self

    def to_dict(self):
        solutions = []
        for solution in self.solutions:
            sol_str = {}
            for key, item in solution.items():
                sol_str[str(key)] = str(item)
            solutions.append(sol_str)

        denoms = []
        for denom in self.denoms:
            denom_str = []
            for item in denom:
                denom_str.append(str(item))
            denoms.append(denom_str)

        bases = []
        for base in self.bases:
            base_str = []
            for item in base:
                base_str.append(str(item))
            bases.append(base_str)

        results = {'scores': [self.score0.abbreviation, self.score1.abbreviation],
                    'score0': self.score0.to_dict(),
                    'score1': self.score1.to_dict(),
                    'solutions': solutions,
                    'non_zero': denoms,
                    'non_negative': bases,
                    #'groebner_basis': [str(gb) for gb in self.gb]
                    }

        return results


In [ ]:
scores = list(base_score_objects.keys())

In [ ]:
results = []

In [ ]:
for idx, score0 in enumerate(scores):
    for score1 in scores[(idx+1):]:
        print(base_score_objects[score0].abbreviation, base_score_objects[score1].abbreviation)
        ps = ProblemSolver2(base_score_objects[score0], base_score_objects[score1])
        ps.solve_()
        ps.edge_cases()
        results.append(ps.to_dict())

acc sens
acc spec
acc ppv
acc npv
acc ba
acc f1p
sens spec
sens ppv
sens npv
sens ba
sens f1p
spec ppv
spec npv
spec ba
spec f1p
ppv npv
ppv ba
ppv f1p
npv ba
npv f1p
ba f1p


In [ ]:
results[1]

{'scores': ['acc', 'spec'],
 'score0': {'abbreviation': 'acc',
  'name': 'accuracy',
  'nans': None,
  'range': [0, 1],
  'synonyms': None,
  'complement': 'error_rate',
  'args': ['tp', 'tn', 'p', 'n'],
  'expression': '(tn + tp)/(n + p)',
  'equation': 'acc - (tn + tp)/(n + p)',
  'equation_polynomial': 'acc*n + acc*p - tn - tp',
  'function': 'accuracy_standardized'},
 'score1': {'abbreviation': 'spec',
  'name': 'specificity',
  'nans': None,
  'range': [0, 1],
  'synonyms': ['selectivity',
   'Selectivity',
   'true_negative_rate',
   'TrueNegativeRate'],
  'complement': 'false_positive_rate',
  'args': ['tn', 'n'],
  'expression': 'tn/n',
  'equation': 'spec - tn/n',
  'equation_polynomial': 'n*spec - tn',
  'function': 'specificity_standardized'},
 'solutions': [{'tn': 'n*spec', 'tp': 'acc*n + acc*p - n*spec'}],
 'non_zero': [[]],
 'non_negative': [[]]}

In [ ]:
import json

In [ ]:
jresults = {'solutions': results}

In [ ]:
with open('solutions.json', 'w') as file:
    json.dump(jresults, file)